# Práctica 1. Resolución de problemas con búsqueda 
## Inteligencia Artificial I    2021/2022

La práctica está organizada en 4 partes:
* Parte I: se muestra a través de ejemplos resueltos cómo se representan algunos problemas clásicos como el de las jarras, el problema de los misioneros o el problema del ocho puzzle. 
* Parte II: se muestra el uso de los algoritmos de búsqueda exhaustiva (ciega y heurística) vistos en clase. 
* Parte III: aprenderemos a medir las propiedades de los algoritmos.
* Parte IV: se os dan 2 problemas algo más complejos para resolver al menos 1.

En el notebook encontraras claramente identificados los lugares en los que debes incluir código o comentarios.  
Cuando termines los ejercicios entrega **este archivo en el campus**.  
Debes incluir al comienzo una celda de markdown con el nombre completo de los miembros del grupo y número de grupo.

Los comentarios razonados de los ejercicios son la parte más importante de esta práctica. 

**Fecha de entrega de la práctica completa: 18 de octubre** 

## Parte I: Representación de problemas de espacios de estados.

Como hemos visto en clase la representación de un problema de espacio de estados consiste en:
* Representar estados y acciones mediante una estructura de datos.
* Definir: estado_inicial, es_estado_final(_), acciones(_), aplica(_,_) y
  coste_de_aplicar_accion, si el problema tiene coste.

 La siguiente clase Problem representa este esquema general de cualquier
 problema de espacio de estados. Un problema concreto será una subclase de
 Problema, y requerirá implementar acciones, aplica y eventualmente __init__, actions,
 goal_test. La función coste_de_aplicar_accion la hemos incluido nosotros
 
El primer paso es importar el código que necesitamos de search.py de AIMA y usar la clase Problem. 

In [1]:
import sys
sys.path.append('aima/')
from search import Problem

In [2]:
## Copiamos aquí la definición de la clase Problem para facilitar la explicación.

class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

Ahora vamos a ver un ejemplo de cómo definir un problema como subclase
de problema. En concreto, el problema de las jarras, visto en clase que es muy sencillo. 

In [3]:
class Jarras(Problem):
    """Problema de las jarras:
    Representaremos los estados como tuplas (x,y) de dos números enteros,
    donde x es el número de litros de la jarra de 4 e y es el número de litros
    de la jarra de 3"""

    def __init__(self):
        self.initial = (0,0)

    def actions(self,estado):
        jarra_de_4=estado[0]
        jarra_de_3=estado[1]
        accs=list()
        if jarra_de_4 > 0:
            accs.append("vaciar jarra de 4")
            if jarra_de_3 < 3:
                accs.append("trasvasar de jarra de 4 a jarra de 3")
        if jarra_de_4 < 4:
            accs.append("llenar jarra de 4")
            if jarra_de_3 > 0:
                accs.append("trasvasar de jarra de 3 a jarra de 4")
        if jarra_de_3 > 0:
            accs.append("vaciar jarra de 3")
        if jarra_de_3 < 3:
            accs.append("llenar jarra de 3")
        return accs

    def result(self,estado,accion):
        j4=estado[0]
        j3=estado[1]
        if accion=="llenar jarra de 4":
            return (4,j3)
        elif accion=="llenar jarra de 3":
            return (j4,3)
        elif accion=="vaciar jarra de 4":
            return (0,j3)
        elif accion=="vaciar jarra de 3":
            return (j4,0)
        elif accion=="trasvasar de jarra de 4 a jarra de 3":
            return (j4-3+j3,3) if j3+j4 >= 3 else (0,j3+j4)
        else: #  "trasvasar de jarra de 3 a jarra de 4"
            return (j3+j4,0) if j3+j4 <= 4 else (4,j3-4+j4)

    def goal_test(self,estado):
        return estado[0]==2


Vamos a probar algunos ejemplos.

In [4]:
p =Jarras()
p.initial

(0, 0)

In [5]:
p.actions(p.initial)

['llenar jarra de 4', 'llenar jarra de 3']

In [6]:
p.result(p.initial,"llenar jarra de 4")

(4, 0)

In [7]:
p.coste_de_aplicar_accion(p.initial,"llenar jarra de 4")

1

In [8]:
p.goal_test(p.initial)

False

### Problema de los misioneros

In [9]:
# Creamos la clase ProblemaMisioneros con los elementos que representarán el problema. 
class ProblemaMisioneros(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal=None):
        '''Inicializacion de nuestro problema.'''
        Problem.__init__(self, initial, goal)
        # cada accion tiene un texto para identificar al operador y despues una tupla de dos elementos con la
        # cantidad de misioneros y canibales que se mueven en la canoa
        self._actions = [('1c', (0, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('2m', (2, 0)), ('1m1c', (1, 1))]

    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        # las acciones validas para un estado son aquellas que al aplicarse
        # nos dejan en otro estado valido
        return [a for a in self._actions if self._is_valid(self.result(s, a))]

    def _is_valid(self, s):
        '''Determina si un estado es valido o no.'''
        # un estado es valido si no hay mas canibales que misioneros en ninguna
        # orilla, y si las cantidades estan entre 0 y 3
        return (s[0] >= s[1] or s[0] == 0) and ((3 - s[0]) >= (3 - s[1]) or s[0] == 3) and (0 <= s[0] <= 3) and (0 <= s[1] <= 3)

    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        # el estado resultante tiene la canoa en el lado opuesto, y con las
        # cantidades de misioneros y canibales actualizadas segun la cantidad
        # que viajaron en la canoa
        if s[2] == 0:
            return (s[0] - a[1][0], s[1] - a[1][1], 1)
        else:
            return (s[0] + a[1][0], s[1] + a[1][1], 0)



In [10]:
# creamos un problema a partir de nuestra formalizacion de ProblemaMisioneros
# como parametros le pasamos el estado inicial, y el estado objetivo que esperamos
misioneros = ProblemaMisioneros((3, 3, 0), (0, 0, 1))

# Asegurate de que entiendes la formalización del problema y haz algunas pruebas con la representación del problema de los misioneros. 
# En la siguiente parte vamos a usar las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura la llamada sería:  
#breadth_first_tree_search(estado).solution()

In [11]:
misioneros.initial

(3, 3, 0)

In [12]:
misioneros.actions(misioneros.initial)

[('1c', (0, 1)), ('2c', (0, 2)), ('1m1c', (1, 1))]

### Representación del problema del puzzle de 8

Vamos a definir la clase Ocho_Puzzle, que implementa la representación del problema del 8-puzzle visto en clase. 
Se os proporciona una versión incompleta y tendréis que completar el código que se presenta a continuación, en los lugares marcados con interrogantes.

### 8 Puzzle 

Tablero 3x3 cuyo objetivo es mover la configuración de las piezas desde un estado inicial dado a un estado objetivo moviendo las fichas al espacio en blanco. 

ejemplo:- 

                  Inicial                             Goal 
              | 7 | 2 | 4 |                       | 1 | 2 | 3 |
              | 5 | 0 | 6 |                       | 4 | 5 | 6 |
              | 8 | 3 | 1 |                       | 7 | 8 | 0 |
              
Hay 9! configuraciones iniciales pero ojo! porque no todas tienen solución. **Tenlo en cuenta al hacer las pruebas**. 

### EJERCICIO 1. Completa la definición de los operadores en el problema del Puzle de 8. 

In [13]:
class Ocho_Puzzle(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self,estado):
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        
        ## vamos comprobando los movimientos que se puede hacer según donde esté el 0
        if pos_hueco not in (0,1,2):
            accs.append("Mover hueco arriba")
        if pos_hueco not in (6,7,8):
            accs.append("Mover hueco abajo")
        if pos_hueco not in (0,3,6):
            accs.append("Mover hueco izquierda")
        if pos_hueco not in (2,5,8):
            accs.append("Mover hueco derecha")
         
        return accs     

    def result(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        ##convertimos estado en una lista 
        ## realizamos la acción de mover el 0
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-3]
            l[pos_hueco-3] = 0        
        if accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco+3]
            l[pos_hueco+3] = 0
        if accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco-1]
            l[pos_hueco-1] = 0
        if accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco+1]
            l[pos_hueco+1] = 0
        
        return tuple(l)
    
    def check_solvability(self, state):
        """ Checks if the given state is solvable """
# The solvability of a configuration can be checked by calculating the Inversion Permutation. 
#If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        return inversion % 2 == 0        
    
    ##def h(self, node):
        ##""" Return the heuristic value for a given state. 
        ##return 1

#### Una vez completada la definición de la clase podrás probar los siguientes ejemplos.

In [14]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [15]:
p8.actions(p8.initial)
#Respuesta: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

In [16]:
p8.result(p8.initial,"Mover hueco arriba")

(2, 8, 3, 1, 0, 4, 7, 6, 5)

In [17]:
p8.result(p8.initial,"Mover hueco abajo")

IndexError: list index out of range

In [18]:
p8.result(p8.initial,"Mover hueco derecha")

(2, 8, 3, 1, 6, 4, 7, 5, 0)

In [19]:
p8.coste_de_aplicar_accion(p8.initial,"Mover hueco abajo")

1

### EJERCICIO 2.  Usando la representación anterior representa el problema del puzle de 16 piezas.

In [20]:
class Quince_Puzzle(Problem): 
    ## Completa la definición del problema
    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 9 , 10 , 11, 12, 13, 14, 15, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self,estado):
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        
        ## vamos comprobando los movimientos que se puede hacer según donde esté el 0
        if pos_hueco not in (0,1,2,3):
            accs.append("Mover hueco arriba")
        if pos_hueco not in (12,13,14,15):
            accs.append("Mover hueco abajo")
        if pos_hueco not in (0,4,8,12):
            accs.append("Mover hueco izquierda")
        if pos_hueco not in (3,7,11,15):
            accs.append("Mover hueco derecha")
         
        return accs     

    def result(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        ##convertimos estado en una lista 
        ## realizamos la acción de mover el 0
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-4]
            l[pos_hueco-4] = 0        
        if accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco+4]
            l[pos_hueco+4] = 0
        if accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco-1]
            l[pos_hueco-1] = 0
        if accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco+1]
            l[pos_hueco+1] = 0
        
        return tuple(l)
    

In [21]:
p16 = Quince_Puzzle((2, 14 ,8, 3, 9, 11, 10 ,1, 6, 4, 7, 0, 12, 13, 15, 5))
p16.initial

(2, 14, 8, 3, 9, 11, 10, 1, 6, 4, 7, 0, 12, 13, 15, 5)

In [22]:
p16.actions(p16.initial)

['Mover hueco arriba', 'Mover hueco abajo', 'Mover hueco izquierda']

In [23]:
p16.result(p16.initial,"Mover hueco derecha")

(2, 14, 8, 3, 9, 11, 10, 1, 6, 4, 7, 12, 0, 13, 15, 5)

## Parte II: Experimentación con los algoritmos implementados. Ejecución de los algoritmos de búsqueda de soluciones para una instancia del Problema.

En primer lugar vamos a ver cómo funciona la búsqueda ciega (en anchura y en profundidad) para encontrar soluciones para los tres problemas anteriores: el problema de las jarras, los misioneros y el problema del ocho puzzle con distintos estados iniciales.

In [24]:
# Cargamos el módulo con los algoritmos de búsqueda.
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

Para que la importación de search funcione el archivo search.py y utils.py tienen que estar en la misma carpeta que este notebook o poner la ruta correspondiente.

In [25]:
# una opcion para poner un prefijo a la ruta antes de importar es:
import sys
sys.path.append('aima-python/')
import search

In [26]:
## comprueba la resolución del problema de las jarras con el método de búsqueda en anchura.  

In [27]:
breadth_first_tree_search(Jarras()).solution()

['llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3',
 'vaciar jarra de 3',
 'trasvasar de jarra de 4 a jarra de 3',
 'llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3']

In [28]:
depth_first_graph_search(Jarras()).solution()

['llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'vaciar jarra de 4',
 'trasvasar de jarra de 3 a jarra de 4']

### EJERCICIO 3. Prueba los algoritmos de búsqueda ciega con el problema de los misioneros y con el  puzzle de 8 y de 16.  Comenta al final cómo se comportan los algoritmos en cuanto a eficiencia y resultados. 

In [29]:
# Usaremos las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. 

#Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura 

breadth_first_tree_search(misioneros).solution()


# Realiza pruebas con los diferentes algoritmos observando el comportamiento.

[('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2))]

In [30]:
depth_first_graph_search(misioneros).solution()

[('1m1c', (1, 1)),
 ('1m', (1, 0)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1m', (1, 0)),
 ('1m1c', (1, 1))]

In [31]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [32]:
p8.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [33]:
breadth_first_tree_search(Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))).solution()
## La llamada corresponde al algoritmo de busqueda en anchura sin control de repetidos.  
## Esta llamada no funciona, se queda en [*]
## Si es método de busqueda en anchura es completo.. ¿no debería terminar? ¿qué crees que está pasando?
## Entonces el estado inicial no se puede resolver, porque el número de inversiones no es par.

KeyboardInterrupt: 

In [34]:
# Comprueba si el estado inicial (2, 8, 3, 1, 6, 4, 7, 0, 5) tiene solución.
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
if p8.check_solvability(p8.initial):
    sol= breadth_first_tree_search(p8).solution()
else: 
    sol="Este estado inicial no tiene solucion"

In [35]:
sol

'Este estado inicial no tiene solucion'

In [36]:
breadth_first_tree_search(Ocho_Puzzle((1,2,3,4,5,6,0,7,8))).solution()

['Mover hueco derecha', 'Mover hueco derecha']

In [37]:
estado = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))

In [38]:
breadth_first_tree_search(estado).solution()
# Respuesta: ['arriba', 'izquierda', 'arriba', 'izquierda', 'abajo', 'derecha', 'derecha', 'abajo']

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [39]:
depth_first_graph_search(estado).solution()

KeyboardInterrupt: 

In [ ]:
breadth_first_graph_search(estado).solution()

In [ ]:
estado = Quince_Puzzle((1,2,3,4,5,6,7,8,9,0,10,11,12,13,14,15))

In [ ]:
breadth_first_tree_search(estado).solution()

In [ ]:
depth_first_graph_search(estado).solution()

In [ ]:
breadth_first_graph_search(estado).solution()

#### En este ejercicio se ha podido observar los resultados y tiempo de la ejecución de los algoritmos de búsqueda ciega.  Escribe aquí tus conclusiones respecto a la resolución de los problemas con búsqueda ciega:
- Misioneros: con ambos algoritmos se hacen el mismo número de viajes.
- Puzle de 8: Las búsquedas de anchura tardan significativamente menos que la búsqueda de profundidad por grafos. Sabemos que la búsqueda en profundidad de grafos termina porque es por grafos pero tarda varios minutos y da una solución con mayor coste
- Puzle de 16:  

**********************************************************************************
Puedes añadir todo el espacio que necesites.

### EJERCICIO 4:  Definición de heurísticas

In [40]:
#### 4.1. Para el problema de los misioneros define una heurística y comenta sus propiedades
#Puedes utilizar alguna de las heurísticas vistas en clase. 

# Heurísticas para el problema de los misioneros.
def h(nodo):
     return 2*(nodo.state[0]+nodo.state[1])-nodo.state[2]
    

### EJERCICIO 5. 
#### Usar las implementaciones de AIMA de los algoritmos de búsqueda_coste_uniforme, busqueda_primero_el_mejor y búsqueda_a_estrella (con las heurísticas anteriores) para resolver el problema de los misioneros y algun ejemplo del puzle de 8 y puzle de 16. 

#### Comparar los costes temporales usando %timeit y comentar los resultados.

In [41]:
misioneros = ProblemaMisioneros((3, 3, 0), (0, 0, 1))
%timeit -n3 astar_search(misioneros, h).solution()

93.1 µs ± 5.77 µs per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [42]:
#### 4.2. Para el problema del puzle de 8 y de 16 se pide definir  las siguientes funciones heurísticas:
# linear(node): cuenta el número de casillas mal colocadas respecto al estado final.
# manhattan(node): suma la distancia Manhattan desde cada casilla a la posición en la que debería estar en el estado final.
# max_heuristic(node: maximo de las dos anteriores
# sqrt_manhattan(node):  raíz cuadrada de la distancia Manhattan

# Heuristicas para el 8 Puzzle. Puedes definir las funciones fuera de la clase ya que en la llamada a A* puedes pasar el nombre 
# de la función. 
import math
def linear(node):
    #goal = node.state.goal
    goal = (1,2,3,4,5,6,7,8,0)
    cont = 0
    for i in goal:
        if goal[i]!=node.state[i]:
            cont=cont+1
    return cont


def manhattan(node):
    state = node.state
    goal = (1,2,3,4,5,6,7,8,0)
    mhd = 0
    for i in goal:
        if state[i] != goal[i]:
            j=0 
            while (node.state[i] != goal[j] and j<9):
                j=j+1
            mhd = mhd + ((i+j) % 3 ) + (abs((i / 3) - (j / 3)))
                                        
    return mhd
                
def sqrt_manhattan(node):
    mhd = manhattan(node)
    
    return math.sqrt(mhd)

def max_heuristic(node):
    score1 = manhattan(node)
    score2 = linear(node)
    return max(score1, score2)

In [43]:
####  Vamos a probar el 8-puzzle utilizando el siguiente **estado inicial** 

#              +---+---+---+
#              | 2 | 4 | 3 |
#              +---+---+---+
#              | 1 | 5 | 6 |
#              +---+---+---+
#              | 7 | 8 | H |
#              +---+---+---+

puzle = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0)) 
%timeit -n3 astar_search(puzle,manhattan).solution()
%timeit -n3 astar_search(puzle,sqrt_manhattan).solution()
%timeit -n3 astar_search(puzle,linear).solution()
%timeit -n3 astar_search(puzle,max_heuristic).solution()

342 µs ± 58.4 µs per loop (mean ± std. dev. of 7 runs, 3 loops each)
843 µs ± 50.4 µs per loop (mean ± std. dev. of 7 runs, 3 loops each)
218 µs ± 55.9 µs per loop (mean ± std. dev. of 7 runs, 3 loops each)
366 µs ± 57.2 µs per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [44]:
astar_search(puzle,manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [45]:
puzle.initial

(2, 4, 3, 1, 5, 6, 7, 8, 0)

In [46]:
puzle.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [47]:
astar_search(puzle,linear).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [48]:
astar_search(puzle,max_heuristic).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

¿Has notado diferencias en los tiempos de ejecución? ¿y en los resultados?
Vamos a medirlo 

Las heuristicas manhattan y max heuristic son las que mas tardan con mucha diferencia

In [49]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
puzzle_2 = Ocho_Puzzle((1, 2, 3, 4, 5, 6, 0, 7, 8))
puzzle_3 = Ocho_Puzzle((1, 2, 3, 4, 5, 7, 8, 6, 0))

In [52]:
%%timeit
astar_search(puzzle_1, linear)
astar_search(puzzle_2, linear)
astar_search(puzzle_3, linear)

KeyboardInterrupt: 

In [ ]:
%%timeit
astar_search(puzzle_1, manhattan)
astar_search(puzzle_2, manhattan)
astar_search(puzzle_3, manhattan)

In [ ]:
%%timeit
astar_search(puzzle_1, sqrt_manhattan)
astar_search(puzzle_2, sqrt_manhattan)
astar_search(puzzle_3, sqrt_manhattan)

In [ ]:
%%timeit
astar_search(puzzle_1, max_heuristic)
astar_search(puzzle_2, max_heuristic)
astar_search(puzzle_3, max_heuristic)

####  Realiza pruebas también con el puzle de 16 con los estados iniciales que quieras.  Escribe aquí tus conclusiones sobre qué heurística te parece mejor para este problema.  

Ten en cuenta que el número de posibles estados iniciales es n!, siendo n el número de fichas (números y hueco). Por tanto, en el puzle 4x4 (16 fichas), tendremos más de 130.000 millones de posibles estados iniciales. Sin embargo, sólo la mitad de esas combinaciones tiene solución. En el caso del siguiente estado inicial que sí tiene solución: (1,2,3,4,5,6,7,8,9,10,12,15,13,14,11,0) indica los nodos generados por A* con alguna de las heurísticas y comparalo con el puzle de 8.


#### Justifica claramente la respuesta basándote en los resultados esperados  (según las clases teóricas) y los resultados obtenidos en las pruebas anteriores.

----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------


## Parte III:  Calcular estadísticas sobre la ejecución de los algoritmos para resolución de problemas de ocho puzzle. 
### El objetivo es comprobar experimentalmente las propiedades teóricas de los algoritmos vistas en clase.
Usaremos la función %timeit para medir los tiempos y para el espacio una version modificada de Problema que almacena el número de nodos.



En la clase modificada también vamos a incluir el calculo que nos diga si un cierto tablero tiene solución o no. Esto es muy útil.. como hemos comentado al principio solo algunos tableros tienen solucion. En el caso del puzle de 8 un tablero se ha demostrado que se puede comprobar calculando su paridad (o número de inversiones). Si es impar el tablero **no** tiene solución.  El concepto de inversión de una ficha será la suma del número de fichas que se encuentran en una posición superior a dicha ficha y que deberían estar situadas en una posición inferior. La inversión total será la suma de las inversiones individuales. Si este número es par, el puzzle tendrá solución. En caso contrario, no habrá solución. 

In [50]:
# Hacemos una definición ampliada de la clase Problem de AIMA que nos va a permitir experimentar con distintos
# estados iniciales, algoritmos y heurísticas. 

# Añadimos en la clase ampliada la capacidad para contar el número de nodos analizados durante la
# búsqueda:


class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)

In [51]:
estado_inicial = (1,2,3,4,5,6,7,0,8)
p8p=Problema_con_Analizados(Ocho_Puzzle(estado_inicial))
p8 = Ocho_Puzzle(estado_inicial)

In [52]:
p8p.initial

(1, 2, 3, 4, 5, 6, 7, 0, 8)

In [53]:
p8p.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [54]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
astar_search(puzzle_1, manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [55]:
#### 4.2. Para el problema del puzle de 8 y de 16 se pide definir  las siguientes funciones heurísticas:
# linear(node): cuenta el número de casillas mal colocadas respecto al estado final.
# manhattan(node): suma la distancia Manhattan desde cada casilla a la posición en la que debería estar en el estado final.
# max_heuristic(node: maximo de las dos anteriores
# sqrt_manhattan(node):  raíz cuadrada de la distancia Manhattan

# Heuristicas para el 8 Puzzle. Puedes definir las funciones fuera de la clase ya que en la llamada a A* puedes pasar el nombre 
# de la función. 
import math
def linear(node):
    #goal = node.state.goal
    goal = (1,2,3,4,5,6,7,8,0)
    cont = 0
    for i in goal:
        if goal[i]!=node.state[i]:
            cont=cont+1
    return cont


def manhattan(node):
    state = node.state
    goal = (1,2,3,4,5,6,7,8,0)
    mhd = 0
    for i,casilla in enumerate(state):
        filaAhora, colAhora = int(i/ 3) , i % 3
        filaGoal,colGoal = int(casilla /3), casilla % 3
        mhd += abs(filaAhora - filaGoal) + abs(colAhora - colGoal)
                                        
    return mhd
                
def sqrt_manhattan(node):
    mhd = manhattan(node)
    
    return math.sqrt(mhd)

def max_heuristic(node):
    score1 = manhattan(node)
    score2 = linear(node)
    return max(score1, score2)
        

In [56]:
astar_search(p8, manhattan).solution()

['Mover hueco derecha']

In [57]:
astar_search(p8p, manhattan).solution()

['Mover hueco derecha']

In [58]:
def resuelve_ocho_puzzle(estado_inicial, algoritmo, h=None):
    """Función para aplicar un algoritmo de búsqueda dado al problema del ocho
       puzzle, con un estado inicial dado y (cuando el algoritmo lo necesite)
       una heurística dada.
       Ejemplo de uso:
           puzzle_1 = (2, 4, 3, 1, 5, 6, 7, 8, 0)
           resuelve_ocho_puzzle(puzzle_1,astar_search,h2_ocho_puzzle)
        Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 
        'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
        Algoritmo: astar_search
        Heurística: h2_ocho_puzzle
        Longitud de la solución: 8. Nodos analizados: 11
       """
    p8=Ocho_Puzzle(estado_inicial)
    p8p=Problema_con_Analizados(p8)
    if p8.check_solvability(estado_inicial):
        if h: 
            sol= algoritmo(p8p,h).solution()
        else: 
            sol= algoritmo(p8p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p8p.analizados))
    else: 
        print("Este problema no tiene solucion. ")


In [59]:
resuelve_ocho_puzzle(estado_inicial,astar_search,manhattan)

Solución: ['Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 1. Nodos analizados: 15


In [60]:
E1 = (2,7,3,4,6,8,1,0,5)
E2 = (3,0,1,8,4,6,5,2,7)
E3 = (4,5,6,1,0,3,7,8,2)
E4 = (1,2,3,0,5,6,4,7,8)
E5 = (0,8,7,6,5,4,3,2,1)
resuelve_ocho_puzzle(E5,astar_search,manhattan)

KeyboardInterrupt: 

### Ejercicio 6:  resolver usando las distintas búsquedas y en su caso, las distintas heurísticas, el problema del 8 puzzle para los siguientes estados iniciales:

E1 = (2,7,3,4,6,8,1,0,5)
E2 = (3,0,1,8,4,6,5,2,7)
E3 = (4,5,6,1,0,3,7,8,2)
E4 = (1,2,3,0,5,6,4,7,8)
E5 = (0,8,7,6,5,4,3,2,1)

          E1              E2              E3              E4               E5
           
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+  
     | 2 | 7 | 3 |   | 3 | 0 | 1 |   | 4 | 5 | 6 |   | 1 | 2 | 3 |    | H | 8 | 7 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+
     | 4 | 6 | 8 |   | 8 | 4 | 6 |   | 1 | 0 | 3 |   | H | 5 | 6 |    | 6 | 5 | 4 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+
     | 1 | H | 5 |   | 5 | 2 | 7 |   | 7 | 8 | 2 |   | 4 | 7 | 8 |    | 3 | 2 | 1 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+
   
 Se pide, en cada caso, obtener detalles del tiempo y espacio necesario para la resolución de estos estados.
 Hacerlo con la función resuelve_ocho_puzzle, para
 obtener, además de la solución, la longitud (el coste) de la solución
 obtenida y el número de nodos analizados. Anotar los resultados en la siguiente tabla (L, longitud de la solución, NA, nodos analizados, T, tiempo)
 
-----------------------------------------------------------------------------------------
                                       E1           E2           E3          E4        E5
    Anchura                            L=           L=           L=          L=        L=
                                       T=           T=           T=          T=        T=
                                       NA=          NA=          NA=         NA=       NA=
   
    Profundidad                        L=           L=           L=          L=        L=
                                       T=           T=           T=          T=        T=
                                       NA=          NA=          NA=         NA=       NA=
                                       
    Coste Uniforme                     L=           L=           L=          L=        L=
                                       T=           T=           T=          T=        T=
                                       NA=          NA=          NA=         NA=       NA=
                                       
    Primero el mejor (linear)          L=           L=           L=          L=        L=
                                       T=           T=           T=          T=        T= 
                                       NA=          NA=          NA=         NA=       NA= 
                                                                            
    Primero el mejor (manhattan)       L=           L=           L=          L= 
                                       T=           T=           T=          T=  
                                       NA=          NA=          NA=         NA=
                                                                             
    A* (linear)                        L=           L=           L=          L= 
                                       T=           T=           T=          T=  
                                       NA=          NA=          NA=         NA=
                                                                             
    A* (manhattan)                     L= 17          L=           L=          L= 
                                       T=           T=           T=          T=  
                                       NA= 200      NA=          NA=         NA=


 -----------------------------------------------------------------------------------------
PUEDES AMPLIAR LA TABLA CON PRUEBAS ADICIONALES. 

Escribe aquí tus conclusiones 

 **justificar los resultados con las distintas propiedades teóricas estudiadas en clase**.  



In [ ]:
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

In [ ]:
E1 = (2,7,3,4,6,8,1,0,5)
E2 = (3,0,1,8,4,6,5,2,7)
E3 = (4,5,6,1,0,3,7,8,2)
E4 = (1,2,3,0,5,6,4,7,8)
E5 = (0,8,7,6,5,4,3,2,1)
resuelve_ocho_puzzle(E5,astar_search,linear)

In [ ]:
E1 = (2,7,3,4,6,8,1,0,5)
E2 = (3,0,1,8,4,6,5,2,7)
E3 = (4,5,6,1,0,3,7,8,2)
E4 = (1,2,3,0,5,6,4,7,8)
E5 = (0,8,7,6,5,4,3,2,1)
resuelve_ocho_puzzle(E1,astar_search,manhattan)


In [ ]:
E1 = (2,7,3,4,6,8,1,0,5)
E2 = (3,0,1,8,4,6,5,2,7)
E3 = (4,5,6,1,0,3,7,8,2)
E4 = (1,2,3,0,5,6,4,7,8)
E5 = (0,8,7,6,5,4,3,2,1)
resuelve_ocho_puzzle(E5,breadth_first_tree_search)

In [ ]:
E1 = (2,7,3,4,6,8,1,0,5)
E2 = (3,0,1,8,4,6,5,2,7)
E3 = (4,5,6,1,0,3,7,8,2)
E4 = (1,2,3,0,5,6,4,7,8)
E5 = (0,8,7,6,5,4,3,2,1)
resuelve_ocho_puzzle(E5,depth_first_graph_search)

In [59]:
E1 = (2,7,3,4,6,8,1,0,5)
E2 = (3,0,1,8,4,6,5,2,7)
E3 = (4,5,6,1,0,3,7,8,2)
E4 = (1,2,3,0,5,6,4,7,8)
E5 = (0,8,7,6,5,4,3,2,1)
resuelve_ocho_puzzle(E5,best_first_graph_search,manhattan)

KeyboardInterrupt: 

### Ejercicio 7:  resolver usando distintas búsquedas y/o heurísticas, el problema del 16 puzzle para algunos  estados iniciales:

Escribe aquí tus conclusiones sobre los resultados y la eficiencia de las distintas opciones. 



###  PARTE IV:  Se os proporcionará un enunciado de algún problema para representar y resolver usando el paradigma del espacio de estados. 





In [628]:

#Funciones auxiliares que dado un conjunto de numeros nos devuelven sus subconjuntos de como mucho tamaño 10 
def subsets(valores):
    if valores == []:
        return [[]]
    x = subsets(valores[1:])
    return x + [[valores[0]] + y for y in x]
 
def subconjuntos_10(valores):
    return [x for x in subsets(valores) if len(x)<=10]


##Funcion heurística.
##Lo que hace es sumar la distancia mínima teórica de cada paquetes a su destino 
##Teniendo en cuenta los siguientes casos, nos da los siguientes valores de distancia:

## Misma ciudad: 1
## Origen y destino aeropuerto y avion en aeropuerto donde el producto : 1
## Origen y destino aeropuerto pero avion no en aeropuerto del producto : 2
## Origen aeropuerto, destino no aeropuerto pero avion en aeropuerto donde el producto: 2
## Resto de casos 3

##Esta heurística no es admisible pues sobreestima
## el coste del camino óptimo, pero conseguir en este problema la optimalidad es complejo.
## Una heurística que no sobreestimaría sería el número de paquetes incorrectamente situados o trabajar también con la
## posición de aviones y camiones pero requeriría cálculos lentos o, como en el caso de los paquetes incorrectos,
## no daría suficiente información y provocaría búsquedas demasiado exhaustivas y potencialmente largas

def heuristic(node, goal, aero):
    s = node.state
    listaProd = s[0]
    cont = 0
    for k,prod in enumerate(listaProd):

        if prod != (-1,-1):
            if prod[0] == goal[k][0]:
                cont=cont+1
            else:
                prodInAero = (prod in aero)
                goalInAero = (goal[k] in aero)
                if(prodInAero): avionInProd = (prod in s[1])
                    
                if (prodInAero and goalInAero):
                    if(avionInProd): cont=cont+1
                    else: cont=cont+2
                    
                elif (prodInAero and avionInProd): cont=cont+2
                    
                elif (goalInAero): cont = cont+2
                else:
                    cont = cont+3
    
    return cont


"""
Función admisible pero muy lenta

"""

def h_admisible(node, goal):
    listaProd= node.state[0]
    aVisitar=set()
    
    
    for k, prod in enumerate(listaProd):
        if prod != (-1,-1):
            aVisitar.add(goal[k])
    return len(aVisitar)
        
    

'''
Con respecto al problema, tenemos varios elementos en el mismo:

1º Las variables asociadas a cada posible estado son:
    Paquetes : tupla de pares, en los que están de forma ordenada (paquete 0, paquete 1, etc) los paquetes
                y en cada par tenemos la ciudad y el almacén 
                Ej: Paquete 1 en ciudad 4 almacen 2 y paquete 2 en ciudad 3 almacen 1 -> 
                        ((4,2),(3,1))
                    Cabe recalcar que los paquetes deberán introducirse de forma ordenada
                    
    Aviones : Tupla de pares en los que se almacenan de forma ordenada la posición de los aviones
                con la misma estructura anterior (Ciudad, Almacen)
    
    Camiones: Tupla que almacena simplemente en qué almacén está cada camión (Introducir camiones de forma ordenada)
                Los camiones no almacenan la ciudad en el estado pues está fija y la guardaremos en una variable
                principal del problema

2º Variables estáticas del problema
    _aeropuertos: La localización en forma de tupla de pares (Ciudad, Almacen) de cada aeropuerto
    
    _ciudadCam:   La ciudad a que hace referencia cada camión en forma de tupla de enteros ordenada 
                  tal y como se ordenaron los camiones
                  
    _almacenesCiudad: Cantidad de almacenes en cada ciudad,en forma de tupla de enteros 
                      y ordenados por el número de la ciudad
    
    goal:         Objetivo de los paquetes, está expresado en el mismo orden y estructura que la variable de estado
                  explicada inicialmente


    
'''

class CiudadesPaquetes(Problem):
    def __init__(self, paquetes, aviones, camiones, aeropuertos, ciudadesCamion, almacenesCiudad,goal):
        '''Inicializacion de nuestro problema.
            Cabe recalcar que primero comprobamos si algún paquete ya ha llegado
            y en ese caso lo marcamos como "enviado" que para nosotros es el estado (-1,-1)        
        '''
        
        for k, prod in enumerate(paquetes):
            if(prod == goal[k]) : paquetes[k]=(-1,-1)

        
        
        Problem.__init__(self, (tuple(paquetes),tuple(aviones),tuple(camiones)), goal)
        self._aeropuertos = aeropuertos
        self._ciudadCam = ciudadesCamion
        self._almaCiudad = almacenesCiudad

        

    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        # las acciones validas para un estado son aquellas que al aplicarse
        # nos dejan en otro estado valido
        accs=list()
        paqs=s[0]
        aviones=s[1]
        trucks=s[2]
        
        '''
        Las acciones válidas se dividen en dos subgrupos:
            1º Acciones de mover aviones:
                Para cada avión, obtenemos todos los paquetes que están en el mismo aeropuerto del avión,
                hacemos todos los posibles subconjuntos de ellos de tamaño maximo 10 
                y añadimos tantas acciones como aeropuertos a los que podría ir con cualquier subconjunto
                de esos paquetes
                
                Ej: 
                
                ("A", 2, [5,2], [1,3,6]) == Mover avión 2 a aeropuerto de la ciudad 5 almacen 2 con paquetes 1  3 y 6
        
            2º Acciones de mover camiones:
                
                Para cada camión miramos qué paquetes están en su ciudad y almacén actual, 
                hacemos los posibles subconjuntos de tamaño máximo 10
                y añadimos tantas acciones como almacenes a los que podríamos llevar ese camión dentro de
                la ciudad.
                
                Ej:
                
                ("T", 4, 3, [3,6,9]) == Mover camión 4 al almacén 3 de su ciudad con los paquetes 3 6 y 9
                
        
        
        '''
        
        for i, avion in enumerate(aviones):
                paquetesValidos = list()
                for k, prod in enumerate(paqs):   
                    if prod == avion:
                        paquetesValidos.append(k+1)
                        
                for j,aero in enumerate(self._aeropuertos):
                        if  avion != aero :
                            for t in subconjuntos_10(paquetesValidos):
                                accs.append(("A",i+1,aero,t))
            
        for i, cam in enumerate(trucks):
                paquetesValidos = list()
                for k,prod in enumerate(paqs):
                        #comprobamos que el paquete esté en la ciudad del camión y en el almacén.
                        if (self._ciudadCam[i] == prod[0] and cam==prod[1]):
                            paquetesValidos.append(k+1)
                        for t in subconjuntos_10(paquetesValidos):
                            for j in range(self._almaCiudad[self._ciudadCam[i]-1]):
                                if (j+1)!=cam:
                                    accs.append(("T",i+1,(j+1),t))

        return accs

    """
        La función de goal simplemente comprueba si hay algún paquete que no esté aún en estado
        "enviado", es decir, que no tenga valor (-1,-1)
    """
    
    def goal_test(self, s):
        for k, prod in enumerate(s[0]):
            if prod!=(-1,-1): return False
            
        return True
    

    def path_cost(self, c, state1, action, state2):
        return 1
     
    

    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.
           
           Simplemente comprueba si es avión o aeropuerto y cambia el estado de dicho avión o aeropuerto
           a su nuevo destino, así como el de los paquetes que se han movido.
           
           De forma simultánea comprobamos si el paquete ha llegado a su destino final y en caso afirmativo 
           lo ponemos a "sent", es decir, (-1,-1)
           
           '''
        
        paqs=list(s[0])
        aviones=list(s[1])
        trucks=list(s[2])
        
        paquetesBien = list()
        #Si la acción es mover avión 
        if(a[0]=="A"):
            aviones[a[1]-1]=a[2] #movemos el avion al aeropuerto que nos mandan
            for k in a[3]:
                paqs[k-1]=a[2]#movemos los paquetes a esa ciudad aeropuerto 
                if(paqs[k-1] == self.goal[k-1]) : paqs[k-1]=(-1,-1) #si el paquete ha llegado ponemos ese valor 
                    
        #si la acción es mover Camión
        else:
            trucks[a[1]-1]= a[2]
            for k in a[3]:
                paqs[k-1]= (paqs[k-1][0],a[2])
                if(paqs[k-1] == self.goal[k-1]) : paqs[k-1]=(-1,-1)
    
        return (tuple(paqs),tuple(aviones),tuple(trucks))
    
       

In [629]:
#Ejemplo a resolver del pdf de la práctica
ciudadesBasico = CiudadesPaquetes([(4,2),(3,1)],[(4,2),(4,2)],[1,1,1,1,1,1],[(1,2),(2,2),(3,2),(4,2),(5,2),(6,2)],[1,2,3,4,5,6],[2,2,2,2,2,2],[(6,2),(1,2)])
ciudades = CiudadesPaquetes([(2,1),(1,2),(1,1),(1,1),(4,2),(3,1)],[(4,2),(4,2)],[1,1,1,1,1,1],[(1,2),(2,2),(3,2),(4,2),(5,2),(6,2)],[1,2,3,4,5,6],[2,2,2,2,2,2],[(2,1),(6,2),(6,1),(3,2),(6,2),(1,2)])
#Ejemplo mucho más complejo y con especificación mas complicada
ciudadesGrande = CiudadesPaquetes([(4,3),(3,1),(2,1),(6,4),(6,1),(3,5),(6,3),(1,2)],[(4,2),(4,2),(6,4)],[1,1,1,1,1,1],[(1,2),(2,2),(3,2),(4,3),(5,2),(6,2),(6,4)],[1,2,3,4,5,6],[5,5,5,5,5,5],[(6,3),(1,2),(2,1),(1,2),(1,1),(1,1),(4,2),(3,1)])

In [630]:
#Vemos que el primer paquete ya estaba en su destino
ciudades.initial


(((-1, -1), (1, 2), (1, 1), (1, 1), (4, 2), (3, 1)),
 ((4, 2), (4, 2)),
 (1, 1, 1, 1, 1, 1))

In [631]:
#Ejemplo de todas las acciones que se pueden hacer nada más empezar el problema
#ciudades.actions(ciudades.initial)

In [632]:
#Ejemplo de hacer actuar un estado 
s1 = ciudades.result(ciudades.initial, ('T', 1, 2, [3]))
s1

(((-1, -1), (1, 2), (1, 2), (1, 1), (4, 2), (3, 1)),
 ((4, 2), (4, 2)),
 (2, 1, 1, 1, 1, 1))

In [633]:
#Esta búsqueda ciega ya sea en anchura con arbol o grafo,puede tardar demasiado porque el número de estados en inabarcable
#breadth_first_tree_search(ciudades).solution()

In [634]:

#Esta otra heurística es admisible pero será más lenta
#astar_search(ciudades, (lambda x:h_admisible(x,ciudades.goal, ciudades._aeropuertos))).solution()
astar_search(ciudadesBasico, (lambda x:h_admisible(x,ciudadesBasico.goal))).solution()

{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(6, 2), (1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2

{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{

{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{

{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{

{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{

...

{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{

{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{

{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{

{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{(1, 2)}
{

[('A', 2, (6, 2), [1]),
 ('A', 1, (1, 2), []),
 ('A', 2, (1, 2), []),
 ('T', 3, 2, [2]),
 ('T', 3, 1, []),
 ('A', 2, (3, 2), []),
 ('A', 2, (1, 2), [2])]

In [635]:
#Este caso es mucho más difícil y por tanto su duracion mucho mayor
#astar_search(ciudadesGrande, (lambda x:heuristic(x,ciudadesGrande.goal, ciudadesGrande._aeropuertos))).solution()

In [626]:
a= Node(ciudadesBasico.initial)
h_admisible(a, ciudadesBasico.goal)

2

In [636]:
astar_search(ciudadesBasico, (lambda x:heuristic(x,ciudadesBasico.goal, ciudadesBasico._aeropuertos))).solution()

[('A', 2, (6, 2), [1]),
 ('A', 1, (1, 2), []),
 ('A', 2, (3, 2), []),
 ('T', 3, 2, [2]),
 ('A', 2, (1, 2), [2])]

In [533]:
astar_search(ciudades, (lambda x:h_admisible(x,h_admisible(x,ciudadesBasico.goal))).solution()

KeyboardInterrupt: 

In [427]:
#Esta solución no tiene por qué ser óptima pero gracias a la heurística  la encontramos en un tiempo factible
astar_search(ciudades, (lambda x:heuristic(x,ciudades.goal, ciudades._aeropuertos))).solution()

[('A', 2, (6, 2), [5]),
 ('A', 1, (1, 2), []),
 ('T', 1, 2, [3, 4]),
 ('A', 1, (6, 2), [2, 3, 4]),
 ('A', 1, (3, 2), [4]),
 ('T', 3, 2, [6]),
 ('A', 1, (1, 2), [6]),
 ('A', 2, (1, 2), []),
 ('T', 1, 1, []),
 ('T', 3, 1, []),
 ('T', 6, 2, []),
 ('T', 6, 1, [3])]